In [39]:
import praw
import pandas as pd
import spacy
from spacy_langdetect import LanguageDetector
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
nlp = spacy.load('en')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)


In [58]:
reddit = praw.Reddit(client_id='kJ8W7GfwB9dz7w', client_secret='t9we10RKcxo28Q90IxZXPPlAWhA', user_agent='Reddit WebScrap')


In [26]:
submission = reddit.submission(id="61vbni")
posts = []
for top_level_comment in submission.comments:
    posts.append(top_level_comment.body)
posts = pd.DataFrame(posts,columns=['Full Text'])


In [29]:
posts = posts[posts['Full Text'] != '[deleted]']
posts

,Full Text
0,I wrote a script that checks my ping to game s...
1,My kids and I used a raspberry pi and some Pyt...
2,web apps are fun and some what cool. I made a ...
3,I got to use Python to automate a lot of the a...
5,I use a Raspberry Pi connected to a servo moto...
...,...
118,I've done [experiments diamond wire cutting ma...
119,I made a subtitle downloader.
120,Google
121,fucking up my life relentlessly


In [40]:
## Deleting Duplicates
posts
def clean_data(dataframe):
    ## Droping Duplicates
    print("Lenght of data before : ", len(dataframe))
    dataframe.drop_duplicates(subset ="Full Text", keep = 'first', inplace = True)
    print("Lenght of data after : ", len(dataframe))

    ##Cleaning Data
    dataframe['Full Text'] = dataframe['Full Text'].str.lower()
    dataframe['Stop_Words'] = ''
    return dataframe

emails_cleand = clean_data(posts)

## Deleting Punctuation


def delete_punct(dataframe):
    tokens_without_punct_spacy= []
    for article in dataframe['Full Text']:
        doc = nlp(article)
        tokens = [t.text for t in doc]
        tokens_without_punct_spacy.append(" ".join([t.text for t in doc if t.pos_ != 'PUNCT']))
    dataframe['Full Text'] = tokens_without_punct_spacy
    return dataframe

emails_no_punct = delete_punct(emails_cleand)

emails_no_punct

## Deleting Stop Words

def stop_words(dataframe):
    list_of_nonStop = []
    for article in dataframe['Full Text']:
        article = nlp(article)
        list_of_nonStop.append(" ".join([notStopWords.text for notStopWords in article if not notStopWords.is_stop]))
    dataframe['Stop_Words'] = list_of_nonStop
    return dataframe

emails_stopss = stop_words(emails_no_punct)

emails_stopss

## Lemmatization

def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

emails_stopss["Lemmatization"] =  emails_stopss.apply(lambda x: lemmatizer(x['Stop_Words']), axis=1)

emails_stopss

## Language Detection

def language(dataframe):
    
    list_of_langs = []
    # sentence level language detection
    index = 0
    for sentence in dataframe['Full Text']:
        doc = nlp(sentence)
        if doc._.language['language']:
            lang= doc._.language['language']
        else:
            lang = ""
        list_of_langs.append(lang)
    print("sdsd", len(list_of_langs))
    dataframe['Lang'] = list_of_langs
    return dataframe

emails_language = language(emails_stopss)
emails_language_eng = emails_language[emails_language['Lang'] == 'en']

Lenght of data before :  119
Lenght of data after :  119
sdsd 119


In [57]:
for post in emails_language_eng['Lemmatization']:
    print("################")
    print(post)

################
write script check ping game server ' m playing get high log router start kick roommates network
################
kids raspberry pi python post facebook time toilet flush call turdtracker 2000
################
web app fun cool little ui webpage raspberry pi pi hook couple custom breadboard led blink temperature & humidity sensor set button webpage signal rpi blink certain lead reading sensor go add camera toggle lead actually snap pic reload page put new image 

   flask
################
get use python automate lot artist bs work like folder structure get access tool / material need start work feature film transformer 3 mib3 immortal ender game green lantern
################
use raspberry pi connect servo motor custom flag monitor company ticket software new incident report need triage flag raise support team aware triage process flag go
################
automate job maybe traditionally cool free time work build drone run analysis making report old fashioned excel     

    
################
decent web scraping think ' ve mention similar thread notable piece work include script daily log dealership ' online listing area ' m anticipate 07 pontiac g6 go forever like inform purchase replacement script run 5 minute baseball season make sure inactive player fantasy lineup
################
' ve get library / program take csv price list product try find amazon find potential project amazon open browser check agree list product ' m try add grab photo description weight bullet point etc create listing ebay newegg amazon jet company website marketplace 

   ' ve create variation pull info ebay manufacturer sites etc amazon step
################
try learn python program automatically care old digimon toy.](https://www.youtube.com / watch?v = xxxqoq16xbu
################
dnd character creator couple month ago     level character save time local convention dnd table set  
################
build framework scrape online serialize novel turn epub
################
wri

need enter lot itune app details wordpress blog api access write script parse itunes destination urls beautiful soup selenium enter datum new post use small python script convert csv invoice ' m consultant nee create lot invoice
################
automate thing python basically job description lately ' ve get bunch tool work farm machine trigger artist check file perforce
################
work city making map map data project recently determine 17,000 street segment report area leave right side street report district boundary run middle street ' m arcpy module great natively include way tell leave right street line 

   write bit code data update script street middle point create point 10 foot leave right midpoint point query layer reporting district feed information line 

   trouble street divide zero roundabout end cul de sacs represent datum teardrop shape loop end point line begin end leave inside loop right outside problem solve neat exercise ' m pretty proud
################
writ


################
give list transaction check -PRON- exist save invoice number status txt file usually 1 minute transaction selenium
################
thing ' ve recently write python script download source html jersey section phoenix suns ' team shop site check black devin booker jersey available     script tweets black jersey find 

   reading feel inspire automate
################
oh boy lot stuff write puppet chef salt start write system deal agentless 

   practice dsl think need stop track use python python eval evil want script run elevate     privilege run js vm inside python effective want 

   finished product 
   basically connect ssh run js script output include script keep redi db get stop js callback 

   work multiprocesse gevent jsonrpc2 interface handle gevent jsonrpc2 wsgi blendin js pydermonkey module run forked process user pretty safe platform limited pydermonkey use v8 need dom parse speed raw js 

   hindsight overshoot get dreamy perspective run stack different l

In [59]:
export_csv = emails_language_eng.to_csv('comments.csv', index = None, header=True)